In [1]:
import pandas as pd
import requests
from time import sleep
from json import JSONDecodeError
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Read authors CSV file
csv_path = 'C:/Users/marcu/Documents/Computational social science/Week 2/Authors.csv'
authors = pd.read_csv(csv_path)

authors

,id,display_name,works_api_url,works_count,country_code,h_index
0,https://openalex.org/A5026829784,Sam Corbett‐Davies,https://api.openalex.org/works?filter=author.i...,34,IL,14
1,https://openalex.org/A5066089123,Byungkyu Lee,https://api.openalex.org/works?filter=author.i...,186,US,13
2,https://openalex.org/A5100435139,Jingwen Zhang,https://api.openalex.org/works?filter=author.i...,399,CN,47
3,https://openalex.org/A5002073039,Sou Hyun Jang,https://api.openalex.org/works?filter=author.i...,69,KR,11
4,https://openalex.org/A5091610280,Carl Colglazier,https://api.openalex.org/works?filter=author.i...,4,US,1
...,...,...,...,...,...,...
1145,https://openalex.org/A5043405308,Yen-Huei Chen,https://api.openalex.org/works?filter=author.i...,29,TW,13
1146,https://openalex.org/A5021090586,Michael Lees,https://api.openalex.org/works?filter=author.i...,226,NL,28
1147,https://openalex.org/A5102918288,Jiayu Zheng,https://api.openalex.org/works?filter=author.i...,35,CN,13
1148,https://openalex.org/A5065295188,Yang Tian,https://api.openalex.org/works?filter=author.i...,450,CN,63


In [3]:
# Split authors into batches (each batch roughly 25 authors)
author_batches = np.array_split(authors, len(authors) // 25)
author_batches[0]

C:\Users\marcu\Documents\Computational social science\venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,id,display_name,works_api_url,works_count,country_code,h_index
0,https://openalex.org/A5026829784,Sam Corbett‐Davies,https://api.openalex.org/works?filter=author.i...,34,IL,14
1,https://openalex.org/A5066089123,Byungkyu Lee,https://api.openalex.org/works?filter=author.i...,186,US,13
2,https://openalex.org/A5100435139,Jingwen Zhang,https://api.openalex.org/works?filter=author.i...,399,CN,47
3,https://openalex.org/A5002073039,Sou Hyun Jang,https://api.openalex.org/works?filter=author.i...,69,KR,11
4,https://openalex.org/A5091610280,Carl Colglazier,https://api.openalex.org/works?filter=author.i...,4,US,1
5,https://openalex.org/A5071422618,Markus Strohmaier,https://api.openalex.org/works?filter=author.i...,348,DE,38
6,https://openalex.org/A5044191812,Vinícius Andrade Brei,https://api.openalex.org/works?filter=author.i...,63,BR,14
7,https://openalex.org/A5019023655,Alyssa Smith,https://api.openalex.org/works?filter=author.i...,4,MX,2
8,https://openalex.org/A5037451955,Alexander Furnas,https://api.openalex.org/works?filter=author.i...,51,US,6
9,https://openalex.org/A5010577211,Dehao Zhang,https://api.openalex.org/works?filter=author.i...,2,CN,1


In [4]:
# Define DataFrame column names
paper_col = ['id', 'publication_year', 'cited_by_count', 'author_ids']
abstract_col = ['id', 'title', 'abstract_inverted_index']

# Global API parameters
source = 'https://api.openalex.org/works'
concepts_1 = {
    'Computer science': 'https://openalex.org/C41008148',
    'Physics': 'https://openalex.org/C121332964',
    'Mathematics': 'https://openalex.org/C33923547',
}
concepts_2 = {
    'Psychology': 'https://openalex.org/C15744967',
    'Sociology': 'https://openalex.org/C144024400',
    'Economics': 'https://openalex.org/C162324750',
    'Political science': 'https://openalex.org/C17744445',
}

In [13]:
def process_batch(author_batch, batch_index):
    """
    Process a single batch of authors by iterating over API pages,
    applying retry logic, and collecting papers and abstracts.
    """
    local_session = requests.Session()  # create a local session for the thread
    local_papers_data = []
    local_abstracts_data = []
    page = 1

    while True:
        tries = 0
        results = None
        # Retry loop for API request
        while tries < 10:
            try:
                # Prepare the filter by joining author ids
                author_ids_str = '|'.join(author_batch['id'].astype(str))
                url = (
                    f"{source}?filter=author.id:{author_ids_str},"
                    f"cited_by_count:>4,cited_by_count:<5001,authors_count:<10,"
                    f"concepts.id:{'|'.join(concepts_1.values())},"
                    f"concepts.id:{'|'.join(concepts_2.values())}"
                )
                response = local_session.get(url, params={'per_page': 200, 'page': page})
                response.raise_for_status()
                results = response.json().get('results', [])
                break  # exit retry loop on success
            except (JSONDecodeError, requests.exceptions.RequestException) as e:
                tries += 1
                sleep(0.1)
        # If no results are returned, exit the paging loop
        if not results:
            break

        # Process each paper in the current page
        for paper in results:
            paper_id = paper['id']
            publication_year = paper.get('publication_year', None)
            cited_by_count = paper.get('cited_by_count', 0)
            # Extract author IDs from the authorships list
            author_ids = ";".join(sub_author['author']['id'] for sub_author in paper['authorships'])
            local_papers_data.append([paper_id, publication_year, cited_by_count, author_ids])

            title = paper.get('title', '')
            abstract_index = paper.get('abstract_inverted_index', None)
            local_abstracts_data.append([paper_id, title, abstract_index])

        page += 1

    print(f"Completed batch {batch_index}")
    return local_papers_data, local_abstracts_data

In [14]:
# Multi-threaded execution using ThreadPoolExecutor
all_papers_data = []
all_abstracts_data = []

# You can adjust max_workers based on your system and API limits
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each batch to be processed in a separate thread
    future_to_batch = {
        executor.submit(process_batch, batch, idx): idx
        for idx, batch in enumerate(author_batches)
    }
    # As each future completes, combine its results
    for future in as_completed(future_to_batch):
        try:
            papers_data, abstracts_data = future.result()
            all_papers_data.extend(papers_data)
            all_abstracts_data.extend(abstracts_data)
        except Exception as exc:
            batch_index = future_to_batch[future]
            print(f"Batch {batch_index} generated an exception: {exc}")

Completed batch 1
Completed batch 4
Completed batch 3
Completed batch 5
Completed batch 9
Completed batch 7
Completed batch 6
Completed batch 2
Completed batch 0
Completed batch 15
Completed batch 11
Completed batch 8
Completed batch 10
Completed batch 16
Completed batch 12
Completed batch 14
Completed batch 19
Completed batch 22
Completed batch 18
Completed batch 17
Completed batch 13
Completed batch 21
Completed batch 26
Completed batch 23
Completed batch 24
Completed batch 25
Completed batch 20
Completed batch 33
Completed batch 30
Completed batch 28
Completed batch 29
Completed batch 34
Completed batch 35
Completed batch 36
Completed batch 32
Completed batch 37
Completed batch 31Completed batch 39

Completed batch 27
Completed batch 38
Completed batch 42
Completed batch 40
Completed batch 44
Completed batch 43
Completed batch 45
Completed batch 41


In [15]:
# Convert the collected data into DataFrames
papers = pd.DataFrame(all_papers_data, columns=paper_col)
abstracts = pd.DataFrame(all_abstracts_data, columns=abstract_col)

In [16]:
# Output final papers DataFrame
print(len(papers), len(abstracts))

17425 17425


In [17]:
papers = papers.drop_duplicates(['id'], ignore_index=True)
papers.to_csv('Works/IC2S2_papers.csv', index=False)
papers

,id,publication_year,cited_by_count,author_ids
0,https://openalex.org/W2018045523,2002,4176,https://openalex.org/A5007285525;https://opena...
1,https://openalex.org/W1987228002,2010,3047,https://openalex.org/A5100744117;https://opena...
2,https://openalex.org/W2142887429,2002,2453,https://openalex.org/A5038976962
3,https://openalex.org/W1972309850,2005,2193,https://openalex.org/A5038976962
4,https://openalex.org/W2018049970,2003,2085,https://openalex.org/A5007285525;https://opena...
...,...,...,...,...
15426,https://openalex.org/W2105361220,2002,5,https://openalex.org/A5030119267;https://opena...
15427,https://openalex.org/W3042189661,1982,5,https://openalex.org/A5007176508
15428,https://openalex.org/W107112977,1997,5,https://openalex.org/A5070348998;https://opena...
15429,https://openalex.org/W2049581058,1991,5,https://openalex.org/A5084116084;https://opena...


In [18]:
abstracts = abstracts.drop_duplicates(['id'], ignore_index=True)
abstracts.to_csv('Works/IC2S2_abstracts.csv', index=False)
abstracts

,id,title,abstract_inverted_index
0,https://openalex.org/W2018045523,Hierarchical Organization of Modularity in Met...,"{'Spatially': [0], 'or': [1], 'chemically': [2..."
1,https://openalex.org/W1987228002,Limits of Predictability in Human Mobility,"{'Predictable': [0], 'Travel': [1], 'Routines'..."
2,https://openalex.org/W2142887429,Linked: The New Science of Networks,"{'The': [0], 'first': [1], 'book': [2, 252], '..."
3,https://openalex.org/W1972309850,The origin of bursts and heavy tails in human ...,None
4,https://openalex.org/W2018049970,Hierarchical organization in complex networks,"{'Many': [0], 'real': [1, 84, 90], 'networks':..."
...,...,...,...
15426,https://openalex.org/W2105361220,Method for evaluating human and simulated driv...,"{'Understanding': [0], 'the': [1, 10, 20, 37, ..."
15427,https://openalex.org/W3042189661,Perception of shape from shading (A),None
15428,https://openalex.org/W107112977,Facial Expression Recognition Using Image Motion,None
15429,https://openalex.org/W2049581058,The Integrated Information Center concept,"{'The': [0, 33], 'Integrated': [1, 45], 'Infor..."


In [29]:
author_ids = ';'.join(papers['author_ids'])
author_ids = author_ids.split(';')

len(author_ids)

57931

In [20]:
author_ids = list(set( author_ids ))
len(author_ids)

22882

In [32]:
author_ids_str = papers['author_ids'][0]
author_ids_list = author_ids_str.split(';')

author_ids_str, author_ids_list

('https://openalex.org/A5007285525;https://openalex.org/A5067021466;https://openalex.org/A5029755266;https://openalex.org/A5088333030;https://openalex.org/A5038976962',
 ['https://openalex.org/A5007285525',
  'https://openalex.org/A5067021466',
  'https://openalex.org/A5029755266',
  'https://openalex.org/A5088333030',
  'https://openalex.org/A5038976962'])